# CV Assignment 1
Pratyay Suvarnapathaki, 2020111016


In [1]:
# imports
import cv2
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np



## Question 1: Camera Calibration using Direct Linear Transform

In [2]:
"""
This block contains all the actual functionality to be implemented.
"""

# estimate the projection matrix using a direct linear transformation
def dlt(loc_img_points, loc_real_points):
	M = []
	for i in range(len(loc_img_points)):
		x, y = loc_img_points[i]
		X, Y, Z = loc_real_points[i]
		M.append([-X, -Y, -Z, -1, 0, 0, 0, 0, x*X, x*Y, x*Z, x])
		M.append([0, 0, 0, 0, -X, -Y, -Z, -1, y*X, y*Y, y*Z, y])
	M = np.array(M)
	u, s, vt = np.linalg.svd(M)
	loc_P = vt[-1, :].reshape(3, 4)
	return loc_P

# project the 3d real points into 2d using the inverse of the projection matrix
def backprojectDLT(loc_real_points, loc_P):
	projected_points = []
	for i in range(len(loc_real_points)):
		X, Y, Z = loc_real_points[i]
		point = np.array([X, Y, Z, 1])
		point = loc_P.dot(point)
		# if(point[2] != 0):
		point = point / point[2]
		projected_points.append(point)
	return projected_points


# visualise the 3d real points and the 2d projected points.
# ground truth = red, reprojected = green
def superimpose2D(exp_num, loc_img, loc_img_points, loc_projected_points):

	temp_img = cv2.imread("../images/Section-1/calib-object.jpg", cv2.IMREAD_COLOR)
	for i in range(len(global_img_points)):
		cv2.circle(temp_img, (int(global_img_points[i][0]), int(
			global_img_points[i][1])), 30, (0, 0, 255), -1)
		
	for i in range(len(loc_projected_points)):
		cv2.circle(temp_img, (int(loc_projected_points[i][0]), int(
			loc_projected_points[i][1])), 30, (0, 255, 0), -1)
	print(len(loc_projected_points)," points reprojected and plotted")
	cv2.imwrite('../results/1_exp'+str(exp_num)+'.png', temp_img)


# P decomposes to K[R|t]
def decomposeP(loc_P):
	loc_K, loc_R = np.linalg.qr(loc_P[:, :3])
	loc_K = loc_K / loc_K[2, 2]
	loc_C = np.linalg.inv(loc_K).dot(loc_P[:, 3])
	return loc_K, loc_R, loc_C

# P = K[R|t]
def recreateP(loc_K, loc_R, loc_C):
	loc_P = np.zeros((3, 4))
	loc_P[:, :3] =loc_K.dot(loc_R)
	loc_P[:, 3] = loc_K.dot(loc_C)
	return loc_P

#plot wireframe
def plotWireframe(exp_num, projections):
	f=plt.figure()
	
	plt.scatter(projections[:, 0], projections[:, 1], s=1, c='b')
	
	img = cv2.imread("../images/Section-1/calib-object.jpg", cv2.IMREAD_COLOR)
	plt.imshow(img)

	# draw the lines
	for i in range(10):
		for j in range(15):
			if i < 9:
				plt.plot([projections[i*15+j, 0], projections[(i+1)*15+j, 0]], [projections[i*15+j, 1], projections[(i+1)*15+j, 1]], 'r', linewidth=0.8)
			if j < 14:
				plt.plot([projections[i*15+j, 0], projections[i*15+j+1, 0]], [projections[i*15+j, 1], projections[i*15+j+1, 1]], 'r', linewidth=0.8)

	plt.savefig('../results/1_wfexp'+str(exp_num)+'.png', dpi=300)
	plt.close(f)

In [3]:
"""
This block contains all the data points. 
"""
global_real_points = [
	# right plane
	[6, -1, 0], [5, -2, 0], [4, -3, 0],
	[3, -4, 0], [2, -5, 0], [1, -6, 0],
	[1, -5, 0], [1, -4, 0], [1, -3, 0], [1, -2, 0],

	# left plane
	[0, -1, 6], [0, -2, 5], [0, -3, 4],
	[0, -4, 3], [0, -5, 2], [0, -6, 1],
	[0, -5, 1], [0, -4, 1], [0, -3, 1], [0, -2, 1]
]

global_img_points = [
	# right plane
    [2415, 1925], [2232, 2086], [2064, 2233],
   	[1918, 2367], [1788, 2486], [1654, 2598],
   	[1661, 2441], [1662, 2279], [1664, 2116], [1666, 1947],
	
	# left plane
   	[630, 1901], [823, 2065], [993, 2217],
   	[1146, 2348], [1289, 2473], [1425, 2596],
   	[1417, 2438], [1418, 2277], [1422, 2109], [1416, 1945]
]

wf_points=[
	[0,0,7], [0,0,6], [0,0,5], [0,0,4], [0,0,3], [0,0,2], [0,0,1],
	[0,0,0], [1,0,0], [2,0,0], [3,0,0], [4,0,0], [5,0,0], [6,0,0], [7,0,0],

	[0,-1,7], [0,-1,6], [0,-1,5], [0,-1,4], [0,-1,3], [0,-1,2], [0,-1,1],
	[0,-1,0], [1,-1,0], [2,-1,0], [3,-1,0], [4,-1,0], [5,-1,0], [6,-1,0], [7,-1,0],

	[0,-2,7], [0,-2,6], [0,-2,5], [0,-2,4], [0,-2,3], [0,-2,2], [0,-2,1],
	[0,-2,0], [1,-2,0], [2,-2,0], [3,-2,0], [4,-2,0], [5,-2,0], [6,-2,0], [7,-2,0],

	[0,-3,7], [0,-3,6], [0,-3,5], [0,-3,4], [0,-3,3], [0,-3,2], [0,-3,1],
	[0,-3,0], [1,-3,0], [2,-3,0], [3,-3,0], [4,-3,0], [5,-3,0], [6,-3,0], [7,-3,0],

	[0,-4,7], [0,-4,6], [0,-4,5], [0,-4,4], [0,-4,3], [0,-4,2], [0,-4,1],
	[0,-4,0], [1,-4,0], [2,-4,0], [3,-4,0], [4,-4,0], [5,-4,0], [6,-4,0], [7,-4,0],

	[0,-5,7], [0,-5,6], [0,-5,5], [0,-5,4], [0,-5,3], [0,-5,2], [0,-5,1],
	[0,-5,0], [1,-5,0], [2,-5,0], [3,-5,0], [4,-5,0], [5,-5,0], [6,-5,0], [7,-5,0],

	[0,-6,7], [0,-6,6], [0,-6,5], [0,-6,4], [0,-6,3], [0,-6,2], [0,-6,1],
	[0,-6,0], [1,-6,0], [2,-6,0], [3,-6,0], [4,-6,0], [5,-6,0], [6,-6,0], [7,-6,0],

	[0,-7,7], [0,-7,6], [0,-7,5], [0,-7,4], [0,-7,3], [0,-7,2], [0,-7,1],
	[0,-7,0], [1,-7,0], [2,-7,0], [3,-7,0], [4,-7,0], [5,-7,0], [6,-7,0], [7,-7,0],

	[0,-8,7], [0,-8,6], [0,-8,5], [0,-8,4], [0,-8,3], [0,-8,2], [0,-8,1],
	[0,-8,0], [1,-8,0], [2,-8,0], [3,-8,0], [4,-8,0], [5,-8,0], [6,-8,0], [7,-8,0],

	[0,-9,7], [0,-9,6], [0,-9,5], [0,-9,4], [0,-9,3], [0,-9,2], [0,-9,1],
	[0,-9,0], [1,-9,0], [2,-9,0], [3,-9,0], [4,-9,0], [5,-9,0], [6,-9,0], [7,-9,0],

]

global_img_points = np.array(global_img_points)
global_real_points = np.array(global_real_points)
wf_points = np.array(wf_points)

In [4]:
"""
In this block I call the implemented functions and print the results for all the experiments.
"""

def performAllTasks(loc_img, img_points, real_points, scale_xy, scale_z, exp_num):
	#scale the real points
	real_points = real_points * np.array([scale_xy, scale_xy, scale_z])
	loc_global_real_points = global_real_points * np.array([scale_xy, scale_xy, scale_z])
	wireframing_points = wf_points * np.array([scale_xy, scale_xy, scale_z])
	

	# TASK A: estimate the projection matrix
	P = dlt(img_points, real_points)

	# backproject the 3d real points into 2d using the projection matrix
	projected_points = backprojectDLT(loc_global_real_points, P)
	
	# TASK B: print the error
	uv_points = np.array(projected_points)[:, :2]
	error = np.average(np.abs(uv_points - global_img_points))
	print("Error:", error)
	
	# TASK C: decompose the projection matrix
	K, R, C = decomposeP(P)
	# verify that P = K[R|t]
	recreatedP = recreateP(K, R, C)
	print(K)
	print(R)
	print("P = K[R|t] verification:", np.allclose(P, recreatedP))

	# TASK D: superimpose the 2d points on the image
	superimpose2D(exp_num, loc_img, img_points, projected_points)

	# make wireframe
	projections = np.array(backprojectDLT(wireframing_points, P))
	plotWireframe(exp_num, projections)




In [5]:
"""
This block contains the various experiments.
"""

# load the image
path1 = "../images/Section-1/calib-object.jpg"
q1img = cv2.imread(path1, cv2.IMREAD_COLOR)
print("Image loaded. Shape:", q1img.shape)

################### EXPERIMENT 1 ###################
exp_num = 1
print("Experiment 1:")
scale_xy_1 = 2.8 #cm/pixel
scale_z_1 = 2.8 #cm/pixel
performAllTasks(q1img, global_img_points, global_real_points, scale_xy_1, scale_z_1, exp_num)

################### EXPERIMENT 2 ###################
exp_num = 2
print("\nExperiment 2:")
scale_xy_2 = 280 #cm/pixel
scale_z_2 = 280 #cm/pixel
performAllTasks(q1img, global_img_points, global_real_points,
                scale_xy_2, scale_z_2, exp_num)

################### EXPERIMENT 3 ###################\
exp_num = 3
print("\nExperiment 3:")
scale_xy_3 = 2.8 #cm/pixel
scale_z_3 = 5.6 #cm/pixel
performAllTasks(q1img, global_img_points, global_real_points,
                scale_xy_3, scale_z_3, exp_num)

################### EXPERIMENT 4 ###################
exp_num = 4
print("\nExperiment 4:")
scale_xy_4 = 2.8 #cm/pixel
scale_z_4 = 2.8 #cm/pixel
exp4_img_points = global_img_points[5:15]
exp4_real_points = global_real_points[5:15]
performAllTasks(q1img, exp4_img_points, exp4_real_points,
                scale_xy_4, scale_z_4, exp_num)

################### EXPERIMENT 5 ###################
exp_num = 5
print("\nExperiment 5:")
scale_xy_5 = 2.8 #cm/pixel
scale_z_5 = 2.8 #cm/pixel
exp5_img_points = global_img_points[6:12]
exp5_real_points = global_real_points[6:12]
performAllTasks(q1img, exp5_img_points, exp5_real_points,
                scale_xy_5, scale_z_5, exp_num)

################### EXPERIMENT 6 ###################
# exp_num = 6
# print("\nExperiment 6:")
# scale_xy_6 = 2.8 #cm/pixel
# scale_z_6 = 2.8 #cm/pixel
# exp6_img_points = global_img_points[10:]
# exp6_real_points = global_real_points[10:]
# performAllTasks(q1img, exp6_img_points, exp6_real_points,scale_xy_6, scale_z_6, exp_num)
# RuntimeWarning: divide by zero encountered in true_divide


Image loaded. Shape: (4160, 3120, 3)
Experiment 1:
Error: 1.6751280611834658
[[-9.04776108e-01 -4.25887548e-01  4.19343876e-04]
 [ 4.25887546e-01 -9.04776201e-01 -9.96316429e-05]
 [ 4.21844196e-04  8.84489957e-05  1.00000000e+00]]
[[ 1.22733484e-02  1.06054539e-02 -2.31135084e-02]
 [ 0.00000000e+00 -2.92053152e-02 -1.62584360e-02]
 [ 0.00000000e+00  0.00000000e+00  1.58968250e-05]]
P = K[R|t] verification: True
20  points reprojected and plotted

Experiment 2:
Error: 1.6752183304877775
[[-9.04747411e-01 -4.25948509e-01  4.19392295e-04]
 [ 4.25948507e-01 -9.04747504e-01 -9.96190476e-05]
 [ 4.21876678e-04  8.85094382e-05  1.00000000e+00]]
[[ 1.22847352e-04  1.06176004e-04 -2.31343568e-04]
 [ 0.00000000e+00 -2.92311825e-04 -1.62760303e-04]
 [ 0.00000000e+00  0.00000000e+00  1.59136258e-07]]
P = K[R|t] verification: True
20  points reprojected and plotted

Experiment 3:
Error: 1.6751163304237195
[[-9.04777456e-01 -4.25884685e-01  4.19341502e-04]
 [ 4.25884682e-01 -9.04777549e-01 -9.9632474

### Report

For experiments 1, 2 and 3, the error is decently low. As can be visually seen in /results, the reprojections are almost perfect -> P is being estimated almost perfectly.
This shows that scale is irrelevant to this problem. This makes intuitive sense as well, as the scale is a constant factor that can be absorbed into the intrinsic matrix K (P is identical for 1 and 2, and differs in the 3rd column between that and the one generated in expt3). Moreover we're using homogenous coordinates, so the scale is irrelevant.

For experiments 4 and 5, the error is much larger due to less points being used. 

For expt 6, the formulation is unsolvable due to the rank being less than 12. This is because the points are coplanar. Code-wise, this results in a runtime error.

## Question 2: DLT with RANSAC


In [6]:
# run ransac
best_inliers=0
inlier_thresh=10
best_points = []


for i in range(100):
	# get 6 random points
	random_points = np.random.choice(range(20), 6)
	random_img_points = global_img_points[random_points]
	random_real_points = global_real_points[random_points]
	
	# estimate the projection matrix
	P = dlt(random_img_points, random_real_points)
	
	# backproject the 3d real points into 2d using the projection matrix
	test_real_points=np.delete(global_real_points,random_points,axis=0)
	test_img_points=np.delete(global_img_points,random_points,axis=0)
	projected_points = backprojectDLT(test_real_points, P)
	
	uv_points = np.array(projected_points)[:, :2]
	new_inliers = len(np.where(np.abs(uv_points - test_img_points) < inlier_thresh)[0])
	if new_inliers > best_inliers:
		best_inliers = new_inliers
		best_points = random_points

# get the best points
best_img_points = global_img_points[best_points]
best_real_points = global_real_points[best_points]

performAllTasks(q1img, best_img_points, best_real_points, scale_xy_1, scale_z_1, "!!Q2!!")
	



/tmp/ipykernel_13172/4125729327.py:26: RuntimeWarning: invalid value encountered in true_divide
  point = point / point[2]
/tmp/ipykernel_13172/4125729327.py:26: RuntimeWarning: divide by zero encountered in true_divide
  point = point / point[2]


Error: 2.4467037250701424
[[-9.66121645e-01 -2.58087157e-01  3.21287279e-04]
 [ 2.58087132e-01 -9.66121698e-01 -1.19670583e-04]
 [ 3.41288032e-04 -3.26962267e-05  1.00000000e+00]]
[[-1.32595978e-02 -4.99838618e-03  2.62595403e-02]
 [ 0.00000000e+00  3.41840697e-02  1.09951838e-02]
 [ 0.00000000e+00  0.00000000e+00 -1.30601320e-05]]
P = K[R|t] verification: True
20  points reprojected and plotted


## Question 3


In [7]:

cubeImage1 = np.array([
[2297, 1559], [2526, 1410], [2709, 1270], [2857, 1168],
[2297, 1912], [2288, 2256], [2292, 2515],
[1961, 1444], [1643, 1338], [1388, 1240],
[2509, 1737], [2662, 1886], [2738, 1980],
[2195, 1304], [2101, 1096], [2038, 960],
[1970, 1793], [1677, 1988], [1464, 2133]])

cubeImage2 = np.array([ 
[1685, 1474], [2029, 1376], [2352, 1274], [2615, 1202],
[1694, 1827], [1732, 2179], [1774, 2460],
[1498, 1347], [1329, 1236], [1205, 1134],
[2042, 1720], [2335, 1933], [2560, 2056],
[1813, 1257], [1932, 1075], [2000, 960],
[1520, 1661], [1380, 1852], [1282, 1967]])

world_points_cube = np.array([
[0, 0, 0], [1, 0, 0], [2, 0, 0], [3, 0, 0],
[0, 1, 0], [0, 2, 0], [0, 3, 0],
[0, 0, 1], [0, 0, 2], [0, 0, 3],
[1, 1, 0], [2, 2, 0], [3, 3, 0],
[1, 0, 1], [2, 0, 2], [3, 0, 3],
[0, 1, 1], [0, 2, 2], [0, 3, 3]
])


In [8]:
# perform DLT to find the projection matrices
P1 = dlt(cubeImage1, world_points_cube)
P2 = dlt(cubeImage2, world_points_cube)

imgpoint1 = [2297, 1559]
imgpoint2 = [1685, 1474]


def triangulate(imgpoint1, imgpoint2, P1, P2):
	A = np.array([
		imgpoint1[1] * P1[2, :] - P1[1, :],
		P1[0, :] - imgpoint1[0] * P1[2, :],
		imgpoint2[1] * P2[2, :] - P2[1, :],
		P2[0, :] - imgpoint2[0] * P2[2, :]
	])

	# solve the linear system
	_, _, V = np.linalg.svd(A)
	point3d = V[-1, :]
	point3d = point3d / point3d[-1]
	return point3d

point3d = triangulate(imgpoint1, imgpoint2, P1, P2)
print("3D point:", point3d)

#euclidean error
error = np.linalg.norm(point3d[:3] - world_points_cube[0])
print("Error:", error)


3D point: [0.02013331 0.02412776 0.06172745 1.        ]
Error: 0.06926598669091223


## Question 4

In [9]:
# paths = ["../images/Section-4/flower/4.jpg",
#          "../images/Section-4/flower/3.jpg",
#          "../images/Section-4/flower/2.jpg",
#          "../images/Section-4/flower/1.jpg"]
# imgname="flower"

# paths = ["../images/Section-4/building/3.jpg",
#          "../images/Section-4/building/2.jpg",
#          "../images/Section-4/building/1.jpg"]
# imgname="building"

paths = ["../images/Section-4/school/2.jpg",
         "../images/Section-4/school/1.jpg"]
imgname="school"

images = [cv2.imread(path) for path in paths]
sift=cv2.xfeatures2d.SIFT_create()
kp=[]
des=[]
for image in images:
	keyp, desc = sift.detectAndCompute(image,None)
	kp.append(keyp)
	des.append(desc)

bf = cv2.BFMatcher()
matches = []
for i in range(len(des)-1):
	matches.append(bf.knnMatch(des[i], des[i+1], k=2))
#ratio test
good = []
for i in range(len(matches)):
	goodmatch=[]
	for m,n in matches[i]:
		if m.distance < 0.8*n.distance:
			goodmatch.append(m)
	good.append(goodmatch)


# # draw matches
# img3 = cv2.drawMatchesKnn(images[0],kp[0],images[1],kp[1],good[0],None,flags=2)
# plt.imshow(img3),plt.show()

# find homography
homographies = []
for i in range(len(good)):
	src_pts = np.float32([ kp[i][m.queryIdx].pt for m in good[i] ]).reshape(-1,1,2)
	dst_pts = np.float32([ kp[i+1][m.trainIdx].pt for m in good[i] ]).reshape(-1,1,2)
	homographies.append(cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)[0])
	

panorama = images[0]
for i in range(len(homographies)):
	panorama = cv2.warpPerspective(panorama, homographies[i], (panorama.shape[1] + images[i+1].shape[1], panorama.shape[0]))
	panorama[0:images[i+1].shape[0], 0:images[i+1].shape[1]] = images[i+1]


# remove black border
gray = cv2.cvtColor(panorama, cv2.COLOR_BGR2GRAY)
_, threshold = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
contours, _ = cv2.findContours(
	threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
x, y, w, h = cv2.boundingRect(contours[0])
cropped_img = panorama[y:y+h, x:x+w]


cv2.imwrite("../results/Q4_"+ imgname + ".jpg", cropped_img)





[ WARN:0@42.130] global shadow_sift.hpp:13 SIFT_create DEPRECATED: cv.xfeatures2d.SIFT_create() is deprecated due SIFT tranfer to the main repository. https://github.com/opencv/opencv/issues/16736


True